In [39]:
!pwd

/content/drive/MyDrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# !gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
# !gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

In [5]:
CONFIG1 = {
    'epoch':10,
    'valid_ratio':0.2,
    'learning_rate':0.01,
    'batch_size':8
}

In [6]:
import math
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset,DataLoader,random_split
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

In [26]:
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
class CovidDataSet(Dataset):
  def __init__(self,x,y):
    self.x,self.y = torch.FloatTensor(x),torch.FloatTensor(y)
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
      return self.x[index],self.y[index]

In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [43]:

train_data=pd.read_csv('./covid.train.csv')
test_data=pd.read_csv('./covid.test.csv')
train_data = train_data.values
test_data = test_data.values
print(test_data.shape)
print(train_data.shape)

(1078, 117)
(2699, 118)


In [10]:
def fsplit(data1):
  ratio_list=[0.8, 0.2]
  generator2 = torch.Generator().manual_seed(42)
  train_data,validation_data = random_split(data1,ratio_list, generator=generator2)
  return np.array(train_data),np.array(validation_data)


In [41]:
train_data , validation_data = fsplit(train_data)


In [42]:
train_data.shape

(2160, 118)

In [44]:
train_data,validation_data,test_data = CovidDataSet(train_data[:,:-1],train_data[:,-1]),\
                    CovidDataSet(validation_data[:,:-1],validation_data[:,-1]),\
                    CovidDataSet(test_data[:,:],test_data[:,-1])
train_data_loader,validation_data_loader,test_data_loader = DataLoader(train_data,256,True),\
                                DataLoader(validation_data,256,True),\
                                DataLoader(test_data,256,True)

In [13]:
device1 = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )

In [32]:
class Model1(nn.Module):
  def __init__(self,feature1):
    super().__init__()
    self.sequence1=nn.Sequential(
            nn.Linear(feature1,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,1)
    )
  def forward(self,input1):
     output1 = self.sequence1(input1)
     return output1.squeeze(1)

In [47]:
def trainer(model1,train_data_loader,val_data_loader,test_data_loader,device1):
  optimizer1 = torch.optim.SGD(model1.parameters(),lr=1e-5,momentum=0.9)
  writer = SummaryWriter()
  min_loss=100
  for i in range(100):
    model1.train()
    loss1_list=0
    step=0
    for x,y in tqdm(train_data_loader,desc='train phase'):
      x,y=x.to(device1),y.to(device1)
      pred1 = model1(x)
      # print(pred1.shape)
      # print(y.shape)
      loss1 = F.mse_loss(pred1,y)
      optimizer1.zero_grad()
      loss1.backward()
      optimizer1.step()
      loss1_list+=loss1.item()
      # print(loss1.item())
      step+=1
    loss1_mean=loss1_list/step
    print('train_loss: {}',loss1_mean)
    model1.eval()
    loss2_list=[]
    for x,y in tqdm(val_data_loader,desc='validation phase'):
      x,y=x.to(device1),y.to(device1)
      with torch.no_grad():
        pred1 = model1(x)
        loss2 = F.mse_loss(pred1,y)
        loss2_list.append(loss2.item())
    loss2_mean=sum(loss2_list)/len(loss2_list)
    print('validation_loss: {}',loss2_mean)
    if loss2_mean < min_loss :
      min_loss=loss2_mean
      torch.save(model1.state_dict(),'model.ckpt')
  x_axse=np.arange(len(loss2_list))
  plt.plot(x_axse,loss2_list)
  plt.ylabel('loss')
  plt.legend('last batch validation')
  plt.show()

In [ ]:
model1=Model1(117).to(device1)
trainer(model1,train_data_loader,validation_data_loader,test_data_loader,device1)

train phase: 100%|██████████| 11/11 [00:00<00:00, 319.27it/s]


train_loss: {} 1068.419472434304


validation phase: 100%|██████████| 3/3 [00:00<00:00, 374.59it/s]


validation_loss: {} 198.43287658691406


train phase: 100%|██████████| 11/11 [00:00<00:00, 315.45it/s]


...

validation phase: 100%|██████████| 3/3 [00:00<00:00, 393.78it/s]


validation_loss: {} 4.001227855682373


train phase: 100%|██████████| 11/11 [00:00<00:00, 238.86it/s]


train_loss: {} 4.5972182317213575


validation phase: 100%|██████████| 3/3 [00:00<00:00, 374.90it/s]


In [45]:
model1=Model1(117).to(device1)
ckpt=torch.load('model.ckpt')
model1.load_state_dict(ckpt)
model1.eval()
for x,_ in test_data_loader:
  x = x.to(device1)
  with torch.no_grad():
    preds=model1(x)
    print(preds)

tensor([ 7.8018, 18.3876, 11.0328, 15.3658, 13.7120, 17.1383,  8.5917, 20.9094,
        17.1640, 16.2036, 11.6450, 19.8835, 21.3650, 15.3877, 16.3537,  7.5093,
        21.9689, 17.4058, 12.8075, 11.3552,  6.4548, 10.3619, 30.8308, 15.3779,
        20.1088,  8.3391, 18.2015, 11.4012, 19.8448, 17.4312, 16.5011, 23.8078,
        21.4560, 10.2038, 16.3934,  8.5593,  7.6142, 19.6596, 13.9899,  8.1010,
        21.5219, 11.4876,  9.7074, 10.8481, 17.1871, 14.3330, 10.3718, 20.0051,
        15.4469,  7.6006, 10.5918, 15.0079,  9.0938, 10.9710, 11.0933, 11.9908,
         7.4969,  9.4260, 15.1932, 16.3673, 11.0245, 15.4046,  9.3126,  7.0391,
        15.0406, 24.7307, 11.5322,  7.4257,  8.8020, 27.1639, 13.3607, 25.1973,
         9.4239, 20.7332, 16.1519, 10.7645,  6.7263, 12.3466, 20.7854, 20.6261,
        14.6700, 31.6709, 11.5488, 11.9092, 25.6686, 13.4217, 15.0875,  9.7306,
        11.2257,  9.5940, 23.2087, 28.2404, 16.7347,  9.0191,  7.5473, 13.1260,
        12.5844, 10.9717, 32.1385,  9.13